In [2]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
df_train = pd.read_csv('train_final.csv')
df_test = pd.read_csv('test_final.csv')

In [4]:
df_test = df_test.drop('ID',axis = 1)

In [5]:
for col in ['workclass', 'education', 'marital.status', 'occupation', 'relationship', 'sex', 'race', 'native.country']:
    one_hot_train = pd.get_dummies(df_train[col], prefix=col)
    one_hot_test = pd.get_dummies(df_test[col], prefix=col)

    df_train = df_train.drop(col,axis = 1)
    df_train = pd.concat([df_train,one_hot_train],axis=1)

    df_test = df_test.drop(col,axis = 1)
    df_test = pd.concat([df_test,one_hot_test],axis=1)


In [6]:
df_test.shape

(23842, 108)

In [7]:
missing_col = None
for col in df_test.columns:
    if col not in df_train.columns:
        df_train[col] = 0
        print(col)

native.country_Holand-Netherlands


In [8]:
df_train

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,income>50K,workclass_?,workclass_Federal-gov,workclass_Local-gov,...,native.country_Puerto-Rico,native.country_Scotland,native.country_South,native.country_Taiwan,native.country_Thailand,native.country_Trinadad&Tobago,native.country_United-States,native.country_Vietnam,native.country_Yugoslavia,native.country_Holand-Netherlands
0,53,93449,15,0,0,40,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,33,123424,13,0,0,40,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,47,144844,9,0,0,40,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,40,114580,9,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,39,115618,9,0,0,50,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,18,83451,9,0,0,25,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
24996,64,202738,9,0,0,35,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
24997,39,225544,14,0,0,40,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24998,53,346871,9,4787,0,46,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [9]:
X = df_train.loc[:, df_train.columns != 'income>50K']
y = df_train['income>50K']


X_test_final = df_test.loc[:, df_test.columns != 'income>50K']

In [10]:
estimator = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

parameters = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05],
    "subsample":[0.5, 0.75, 1],
    "min_child_weight":[1,5,15]
}

In [11]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'roc_auc',
    n_jobs = 10,
    cv = 1,
    verbose=True
)

In [12]:
grid_search.fit(X,y)

Fitting 10 folds for each of 864 candidates, totalling 8640 fits


KeyboardInterrupt: 

In [ ]:
print(" Results from Grid Search " )
print("\n The best estimator across ALL searched params:\n",grid_search.best_estimator_)
print("\n The best score across ALL searched params:\n",grid_search.best_score_)
print("\n The best parameters across ALL searched params:\n",grid_search.best_params_)

In [107]:
model = XGBClassifier(eta='0.1', subsample=0.8)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False, eta='0.1',
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.100000001, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, ...)

In [108]:
y_pred = model.predict(X_test)

In [81]:
y_pred.shape

(25000,)

In [109]:
import csv
ctr = 1
data = [['ID', 'Prediction']]
for pred in y_pred:
    row = [ctr, pred]
    data.append(row)
    ctr += 1
# open the file in the write mode
with open('results.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(data)

In [132]:
w = np.asarray([1,1,0])
x=np.asarray([0,1,1])
w@x

1

In [133]:
1 / np.linalg.norm(w)

0.7071067811865475